# Импорты

In [1]:
import pandas as pd
import json # возможно потом понадобится для загрузки словарей
from collections import Counter
import pickle

# Загрузка наборов данных

In [2]:
# зададим директорию откуда будет забирать тренировочный набор даннх
file_path_train = 'train.json'

In [3]:
# загружаем наш тренировочный набор данных в pandas
df_train = pd.read_json(file_path_train)

In [4]:
# смотрим сверху - ужас наши юзеры в колонках мы так не привыкли
df_train.head()

,user_1,user_2,user_3,user_4,user_5,user_6,user_7,user_8,user_9,user_10,...,user_127746,user_127747,user_127748,user_127749,user_127750,user_127751,user_127752,user_127753,user_127754,user_127755
target,female,female,male,male,female,female,female,female,female,male,...,male,female,male,female,male,male,male,female,female,female
features,"{'orders': [{'site-id': 1, 'orders': [{'create...","{'visits': [{'site-id': 3, 'first-seen': 16962...","{'orders': [{'site-id': 21, 'orders': [{'creat...","{'orders': [{'site-id': 2, 'orders': [{'create...","{'orders': [{'site-id': 39, 'orders': [{'creat...","{'orders': [{'site-id': 123, 'orders': [{'crea...","{'visits': [{'site-id': 225, 'first-seen': 169...","{'orders': [{'site-id': 196, 'orders': [{'crea...","{'orders': [{'site-id': 307, 'orders': [{'crea...","{'orders': [{'site-id': 49, 'orders': [{'creat...",...,"{'orders': [{'site-id': 173, 'orders': [{'crea...","{'visits': [{'site-id': 3, 'first-seen': 16897...","{'orders': [{'site-id': 348, 'orders': [{'crea...","{'orders': [{'site-id': 39, 'orders': [{'creat...","{'orders': [{'site-id': 41, 'orders': [{'creat...","{'orders': [{'site-id': 307, 'orders': [{'crea...","{'orders': [{'site-id': 147, 'orders': [{'crea...","{'orders': [{'site-id': 20, 'orders': [{'creat...","{'orders': [{'site-id': 407, 'orders': [{'crea...","{'orders': [{'site-id': 726, 'orders': [{'crea..."


Отмечам про себя что в наборе у нас 127755 пользователей

In [5]:
# транспонируем датасет что бы он принял человеческий вид
df_train = df_train.T

# зададим имена колонок (важно когда будем работать с тестовым набором то колонки таргет у нас не будет)
df_train.reset_index(inplace=True)
df_train.rename(columns={'index': 'user_id', 0: 'target', 1: 'features'}, inplace=True)

# Посмотрим что получилось
df_train.head()


,user_id,target,features
0,user_1,female,"{'orders': [{'site-id': 1, 'orders': [{'create..."
1,user_2,female,"{'visits': [{'site-id': 3, 'first-seen': 16962..."
2,user_3,male,"{'orders': [{'site-id': 21, 'orders': [{'creat..."
3,user_4,male,"{'orders': [{'site-id': 2, 'orders': [{'create..."
4,user_5,female,"{'orders': [{'site-id': 39, 'orders': [{'creat..."


Далее мы должны распарсить поле характеристики, там все самое интересное, вот этим и займемся ниже

In [6]:
# найдем все уникальные ключи внути ключа features, что бы понять сколько в нем смысловых блоков из которым мы будем парсить данные
unique_keys = set()
for features in df_train['features']:
    unique_keys.update(features.keys()) # просто вытащим их в форме множества

# создадим из них новые колонки извлекая данные из 'features'
for key in unique_keys:
    df_train[key] = df_train['features'].apply(lambda x: x.get(key)) # да просто обратимся по ключу

# колонку 'features' удалим за дальнейшей ненадобностью
df_train.drop('features', axis=1, inplace=True)

# посмотрим что получилось
df_train.head()


,user_id,target,orders,exchange-sessions,last-visits-in-categories,site-meta,visits
0,user_1,female,"[{'site-id': 1, 'orders': [{'created-at': 1634...",None,None,[{'site-id': 2}],None
1,user_2,female,None,None,"[{'category': 'other', 'last-visit-at': 169627...","[{'site-id': 4}, {'site-id': 5}, {'site-id': 6...","[{'site-id': 3, 'first-seen': 1696277805, 'las..."
2,user_3,male,"[{'site-id': 21, 'orders': [{'created-at': 168...",None,"[{'category': 'electronics', 'last-visit-at': ...","[{'site-id': 24}, {'site-id': 25}, {'site-id':...","[{'site-id': 3, 'first-seen': 1696320204, 'las..."
3,user_4,male,"[{'site-id': 2, 'orders': [{'created-at': 1600...",None,"[{'category': 'hypermarket', 'last-visit-at': ...","[{'site-id': 33}, {'site-id': 2}, {'site-id': ...","[{'site-id': 3, 'first-seen': 1693468067, 'las..."
4,user_5,female,"[{'site-id': 39, 'orders': [{'created-at': 164...",None,"[{'category': 'insurance', 'last-visit-at': 16...","[{'site-id': 42}, {'site-id': 43}, {'site-id':...","[{'site-id': 3, 'first-seen': 1693796766, 'las..."


![Описание наших колонок с данными](03ed856b-a2a1-423c-a79a-635b2a9fd5ba.jpg)


## Ну что же приступаем к первому этапу извлечения данных, сначала разберемся с заказами и вытащим из них сайты

In [7]:
# Функция рассчитывает определенные айди сайтов в колонке заказы посчитает сколько
# раз наш пользователь заказывал что то на этом сайте
def count_site_ids_in_orders_modified(orders):
    site_id_counter = Counter()
    if orders is not None:
        for order_dict in orders:
            if 'orders' in order_dict and 'site-id' in order_dict:
                site_id = order_dict['site-id']
                for order in order_dict['orders']:
                    site_id_counter[site_id] += 1
    return site_id_counter

In [8]:

# Создадим новую колонку в которой будет количество заказов на каждом сайте для каждого пользователя
df_train['orders_count'] = df_train['orders'].apply(count_site_ids_in_orders_modified)

# посмотрим что получилось
df_train[['user_id', 'orders_count']].head()


,user_id,orders_count
0,user_1,"{1: 1, 2: 2}"
1,user_2,{}
2,user_3,"{21: 1, 22: 1}"
3,user_4,"{2: 1, 29: 2, 30: 4, 21: 1}"
4,user_5,"{39: 1, 40: 1, 20: 1}"


In [9]:
# теперь тоже самое сделаем и для колонки с посещениями сайтов - посмотрим куда человек
#  ходит и как часто
def count_site_ids_in_visits_modified(visits):
    site_id_counter = Counter()
    if visits is not None:
        for visit_group in visits:
            if 'visits' in visit_group and 'site-id' in visit_group:
                site_id = visit_group['site-id']
                for visit in visit_group['visits']:
                    site_id_counter[site_id] += 1
    return site_id_counter

In [10]:

# Применим функцию и создадим новый столбец с посещениями сайтов
df_train['visits_count'] = df_train['visits'].apply(count_site_ids_in_visits_modified)

# посмотрим что получилось
df_train[['user_id', 'visits_count']].head()


,user_id,visits_count
0,user_1,{}
1,user_2,{3: 2}
2,user_3,"{3: 1, 23: 3, 16: 10}"
3,user_4,"{3: 2, 21: 1, 20: 4, 31: 1, 30: 6, 32: 3}"
4,user_5,"{3: 3, 23: 1, 13: 7, 41: 2, 20: 10}"


In [11]:
# теперь для каждого пользвателя мы хотим вытащить последние категории посещений
# они не повторяются поэтому нам не нужен словарь обойдемся списком
def extract_categories(last_visits):
    if last_visits is not None:
        return [visit['category'] for visit in last_visits]
    return []

In [12]:
# применим функцию и создадим новый столбец с требуемым списком
df_train['last_visited_categories'] = df_train['last-visits-in-categories'].apply(extract_categories)

# И сразу проверим что получилось
df_train[['user_id', 'last_visited_categories']].head()

,user_id,last_visited_categories
0,user_1,[]
1,user_2,[other]
2,user_3,"[electronics, other]"
3,user_4,"[hypermarket, other, education]"
4,user_5,"[insurance, fashion, other, bank, pets, electr..."


In [13]:
# Функция для извлечения списка site-id из колонки site-meta
# описание довольно размытое, но мета в dote это те герои которые популярны сейчас,
# в текущих реалиях, возмлжно это сайты которые сейчас популярны у пользователя
# это важно стоит потом подумать о весе этого списка (да снова список)
def extract_site_meta(site_meta):
    if site_meta is not None:
        return [meta['site-id'] for meta in site_meta]
    return []


In [14]:
# Применение функции к столбцу site-meta
df_train['site_meta_list'] = df_train['site-meta'].apply(extract_site_meta)
# проверим что там у нас получилось
df_train[['user_id', 'site_meta_list']].head()

,user_id,site_meta_list
0,user_1,[2]
1,user_2,"[4, 5, 6, 7, 8, 9, 10, 11, 12]"
2,user_3,"[24, 25, 26, 21, 16, 17, 27, 28, 22]"
3,user_4,"[33, 2, 34, 35, 30, 32, 21, 36, 20, 29, 17, 3,..."
4,user_5,"[42, 43, 44, 35, 41, 36, 39, 20, 45, 46, 17, 3..."


Мы незаслуженно обошли внииманием дополнительную информацию, которая есть в колонке заказы!!! Нужно это срочно устранить

In [15]:
# эта функция посччитает количество предметов купленных пользователем и сгруппирует их
# по их номеру
def count_item_ids_in_orders(orders):
    item_id_counter = Counter()
    if orders is not None:
        for order_dict in orders:
            for order in order_dict.get('orders', []):
                for item in order.get('items', []):
                    item_id = item.get('id')
                    count = item.get('count', 1)  # Устанавливаем значение по умолчанию 1
                    if count is not None:  # Дополнительная проверка на None
                        item_id_counter[item_id] += count
    return item_id_counter


In [16]:
# эта функция в том же накопительном режиме спарсит категории товаров, которые
# покупал пользователь
def count_categories_in_orders(orders):
    category_counter = Counter()
    if orders is not None:
        for order_dict in orders:
            for order in order_dict.get('orders', []):
                for item in order.get('items', []):
                    categories = item.get('general-category-path', [])
                    for category in categories:
                        category_counter[category] += 1
    return category_counter

In [17]:
# Эта функция займется подсчетом бредов, тоже показтельно
def count_brand_ids_in_orders(orders):
    brand_id_counter = Counter()
    if orders is not None:
        for order_dict in orders:
            for order in order_dict.get('orders', []):
                for item in order.get('items', []):
                    brand_id = item.get('brand-id')
                    count = item.get('count', 1)  # Устанавливаем значение по умолчанию 1
                    if brand_id is not None and count is not None:  # Дополнительная проверка на None
                        brand_id_counter[brand_id] += count
    return brand_id_counter


In [18]:
# Применение функции к заказам - вытащим товры
df_train['item_ids_count'] = df_train['orders'].apply(count_item_ids_in_orders)
df_train[['user_id', 'item_ids_count']].head()

,user_id,item_ids_count
0,user_1,{}
1,user_2,{}
2,user_3,{'item_1': 1}
3,user_4,"{'item_8': 1, 'item_9': 1, 'item_10': 1, 'item..."
4,user_5,"{'item_13': 1, 'item_14': 1, 'item_15': 1, 'it..."


In [19]:
# тут выащим категорию товара (! обязательно потом join с табличкой глазами глянуть)
df_train['category_path_count'] = df_train['orders'].apply(count_categories_in_orders)
df_train[['user_id', 'category_path_count']].head()

,user_id,category_path_count
0,user_1,{}
1,user_2,{}
2,user_3,{}
3,user_4,"{90404: 1, 90403: 1, 90402: 1, 90698: 1, 73303..."
4,user_5,"{91491: 1, 91461: 2, 198119: 2, 91072: 1, 9149..."


In [20]:
# ну и бренды, на самом деле они мне кажется мало информативны, но все таки посмотрим
df_train['brand_ids_count'] = df_train['orders'].apply(count_brand_ids_in_orders)
df_train[['user_id', 'brand_ids_count']].head()

,user_id,brand_ids_count
0,user_1,{}
1,user_2,{}
2,user_3,{}
3,user_4,"{3243: 1, 1210537: 1, 1773: 1}"
4,user_5,"{1: 1, 1161: 1, 4315: 1}"


У нас есть еще одна загадочная колонка с предыдущим опытом игры в витрину подарков, это может быть интересно, потому что если я правильно понял, то человек из интересного ему выбирал самое интересное!

In [21]:
# Функция для извлечения списка выбранных сайтов из колонки exchange-sessions
def extract_selected_sites(exchange_sessions):
    selected_sites = []
    if exchange_sessions is not None:
        for session in exchange_sessions:
            accepted_site_id = session.get('accepted-site-id')
            if accepted_site_id is not None:
                selected_sites.append(accepted_site_id)
    return selected_sites


In [22]:
# Применение функции к DataFrame, вытащим что же наши пользователи выбирали
df_train['selected_sites'] = df_train['exchange-sessions'].apply(extract_selected_sites)

In [23]:
# Проверка результата для первых нескольких пользователей
print(df_train[['user_id', 'selected_sites']].tail())

            user_id selected_sites
127750  user_127751             []
127751  user_127752             []
127752  user_127753             []
127753  user_127754           [93]
127754  user_127755             []


In [24]:
# посмотрим на набор колонок, который у нас получился
df_train.columns

Index(['user_id', 'target', 'orders', 'exchange-sessions',
       'last-visits-in-categories', 'site-meta', 'visits', 'orders_count',
       'visits_count', 'last_visited_categories', 'site_meta_list',
       'item_ids_count', 'category_path_count', 'brand_ids_count',
       'selected_sites'],
      dtype='object')

In [25]:
# удалим колонки исходники за ненадобнотстью:)
df_train = df_train.drop(['visits', 'orders', 'last-visits-in-categories',
       'exchange-sessions', 'site-meta'], axis=1)

In [26]:

# посмотрим на на результат нашего features engeneering
df_train.head()


,user_id,target,orders_count,visits_count,last_visited_categories,site_meta_list,item_ids_count,category_path_count,brand_ids_count,selected_sites
0,user_1,female,"{1: 1, 2: 2}",{},[],[2],{},{},{},[]
1,user_2,female,{},{3: 2},[other],"[4, 5, 6, 7, 8, 9, 10, 11, 12]",{},{},{},[]
2,user_3,male,"{21: 1, 22: 1}","{3: 1, 23: 3, 16: 10}","[electronics, other]","[24, 25, 26, 21, 16, 17, 27, 28, 22]",{'item_1': 1},{},{},[]
3,user_4,male,"{2: 1, 29: 2, 30: 4, 21: 1}","{3: 2, 21: 1, 20: 4, 31: 1, 30: 6, 32: 3}","[hypermarket, other, education]","[33, 2, 34, 35, 30, 32, 21, 36, 20, 29, 17, 3,...","{'item_8': 1, 'item_9': 1, 'item_10': 1, 'item...","{90404: 1, 90403: 1, 90402: 1, 90698: 1, 73303...","{3243: 1, 1210537: 1, 1773: 1}",[]
4,user_5,female,"{39: 1, 40: 1, 20: 1}","{3: 3, 23: 1, 13: 7, 41: 2, 20: 10}","[insurance, fashion, other, bank, pets, electr...","[42, 43, 44, 35, 41, 36, 39, 20, 45, 46, 17, 3...","{'item_13': 1, 'item_14': 1, 'item_15': 1, 'it...","{91491: 1, 91461: 2, 198119: 2, 91072: 1, 9149...","{1: 1, 1161: 1, 4315: 1}",[]


In [27]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127755 entries, 0 to 127754
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   user_id                  127755 non-null  object
 1   target                   127755 non-null  object
 2   orders_count             127755 non-null  object
 3   visits_count             127755 non-null  object
 4   last_visited_categories  127755 non-null  object
 5   site_meta_list           127755 non-null  object
 6   item_ids_count           127755 non-null  object
 7   category_path_count      127755 non-null  object
 8   brand_ids_count          127755 non-null  object
 9   selected_sites           127755 non-null  object
dtypes: object(10)
memory usage: 9.7+ MB


Проведем такие же преобразования и для валидационного набора данных

In [28]:
df_val = pd.read_json('val.json')

In [29]:
# транспонируем датасет что бы он принял человеческий вид
df_val = df_val.T

# зададим имена колонок (важно когда будем работать с тестовым набором то колонки таргет у нас не будет)
df_val.reset_index(inplace=True)
df_val.rename(columns={'index': 'user_id', 0: 'target', 1: 'features'}, inplace=True)

# Посмотрим что получилось
df_val.head()

,user_id,target,features
0,user_127756,female,"{'orders': [{'site-id': 407, 'orders': [{'crea..."
1,user_127757,male,"{'orders': [{'site-id': 16, 'orders': [{'creat..."
2,user_127758,female,"{'orders': [{'site-id': 149, 'orders': [{'crea..."
3,user_127759,female,"{'orders': [{'site-id': 93, 'orders': [{'creat..."
4,user_127760,female,"{'visits': [{'site-id': 3, 'first-seen': 16836..."


In [30]:
# найдем все унникальные ключи внути ключа features
unique_keys = set()
for features in df_val['features']:
    unique_keys.update(features.keys())

# создадим из них новые колонки извлекая данные из 'features'
for key in unique_keys:
    df_val[key] = df_val['features'].apply(lambda x: x.get(key))

# колонку 'features' удалим за дальнейшей ненадобностью
df_val.drop('features', axis=1, inplace=True)

# посмотрим что получилось
df_val.head()

,user_id,target,orders,exchange-sessions,last-visits-in-categories,site-meta,visits
0,user_127756,female,"[{'site-id': 407, 'orders': [{'created-at': 16...","[{'landed-at': 1697132982, 'sites': [169, 214,...","[{'category': 'other', 'last-visit-at': 169713...",[{'site-id': 407}],"[{'site-id': 3, 'first-seen': 1697125590, 'las..."
1,user_127757,male,"[{'site-id': 16, 'orders': [{'created-at': 163...","[{'landed-at': 1693508972, 'sites': [2, 169, 7...","[{'category': 'electronics', 'last-visit-at': ...","[{'site-id': 16, 'recency': 1, 'frequency': 5,...","[{'site-id': 3, 'first-seen': 1698423484, 'las..."
2,user_127758,female,"[{'site-id': 149, 'orders': [{'created-at': 16...",None,"[{'category': 'sport', 'last-visit-at': 165060...","[{'site-id': 391}, {'site-id': 42}, {'site-id'...","[{'site-id': 391, 'first-seen': 1685615371, 'l..."
3,user_127759,female,"[{'site-id': 93, 'orders': [{'created-at': 162...","[{'landed-at': 1697791749, 'sites': [373, 169,...","[{'category': 'furniture', 'last-visit-at': 16...","[{'site-id': 93, 'recency': 5, 'frequency': 2,...","[{'site-id': 495, 'first-seen': 1687325784, 'l..."
4,user_127760,female,None,None,"[{'category': 'other', 'last-visit-at': 169765...","[{'site-id': 46}, {'site-id': 29}, {'site-id':...","[{'site-id': 3, 'first-seen': 1683626530, 'las..."


In [31]:
# Создадим новую колонку в которой будет количество заказов на каждом сайте для каждого пользователя
df_val['orders_count'] = df_val['orders'].apply(count_site_ids_in_orders_modified)

# Применим функцию и создадим новый столбец с посещениями сайтов
df_val['visits_count'] = df_val['visits'].apply(count_site_ids_in_visits_modified)

# применим функцию и создадим новый столбец с требуемым списком по категориям последних посещений
df_val['last_visited_categories'] = df_val['last-visits-in-categories'].apply(extract_categories)

# Применение функции к столбцу site-meta
df_val['site_meta_list'] = df_val['site-meta'].apply(extract_site_meta)

# Применение функции к заказам - вытащим товры
df_val['item_ids_count'] = df_val['orders'].apply(count_item_ids_in_orders)

# тут выащим категорию товара (! обязательно потом join с табличкой глазами глянуть)
df_val['category_path_count'] = df_val['orders'].apply(count_categories_in_orders)

# ну и бренды, на самом деле они мне кажется мало информативны, но все таки посмотрим
df_val['brand_ids_count'] = df_val['orders'].apply(count_brand_ids_in_orders)

# Применение функции к DataFrame, вытащим что же наши пользователи выбирали
df_val['selected_sites'] = df_val['exchange-sessions'].apply(extract_selected_sites)


In [32]:
# удалим колонки исходники за ненадобнотстью:)
df_val = df_val.drop(['visits', 'orders', 'last-visits-in-categories',
       'exchange-sessions', 'site-meta'], axis=1)

In [33]:
# посмотрим как обработался валидационный набор
df_val.head()

,user_id,target,orders_count,visits_count,last_visited_categories,site_meta_list,item_ids_count,category_path_count,brand_ids_count,selected_sites
0,user_127756,female,{407: 1},"{3: 2, 407: 10}","[other, education]",[407],{},{},{},"[262, 169]"
1,user_127757,male,"{16: 10, 40: 1, 288: 1, 21: 2}","{3: 1, 308: 1, 16: 1}","[electronics, other, shoes]","[16, 29, 21, 40, 212]","{'item_3780263': 2, 'item_5403882': 2, 'item_1...","{90578: 3, 90574: 3, 198118: 5, 91498: 1, 9149...","{1237: 5, 1: 2, 3519: 1, 996: 1, 1055: 3, 1650...",[42]
2,user_127758,female,"{149: 2, 25: 1, 21: 2, 100: 1, 474: 1}","{391: 1, 42: 7, 485: 1, 820: 1, 97: 5, 39: 1, ...","[sport, furniture, fashion, other, bank, house...","[391, 42, 485, 25, 149, 21, 100, 398, 54, 17, ...","{'item_5409213': 1, 'item_1718311': 2, 'item_1...","{13243353: 1, 91182: 1, 91157: 1, 90509: 4, 76...","{62479: 1, 16627: 4, 595: 2, 9851: 4, 50651: 2...",[]
3,user_127759,female,"{93: 8, 134: 1, 103: 1}","{495: 1, 34: 9, 484: 1, 334: 1, 205: 1, 31: 1,...","[furniture, fashion, other, food, electronics,...","[93, 348, 495, 103, 134, 42, 17, 68]","{'item_508120': 1, 'item_4523963': 1, 'item_93...","{4854062: 3, 91182: 5, 91157: 20, 90509: 20, 8...","{3192: 23, 7066: 1, 3860: 1}","[205, 131]"
4,user_127760,female,{},"{3: 2, 34: 1}","[other, cosmetics]","[46, 29, 34, 12, 93, 42, 76, 17]",{},{},{},[]


In [34]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27447 entries, 0 to 27446
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   user_id                  27447 non-null  object
 1   target                   27447 non-null  object
 2   orders_count             27447 non-null  object
 3   visits_count             27447 non-null  object
 4   last_visited_categories  27447 non-null  object
 5   site_meta_list           27447 non-null  object
 6   item_ids_count           27447 non-null  object
 7   category_path_count      27447 non-null  object
 8   brand_ids_count          27447 non-null  object
 9   selected_sites           27447 non-null  object
dtypes: object(10)
memory usage: 2.1+ MB


Загрузим тестовый надор данных

In [35]:
test_path = 'test.json'

In [36]:
# нам понадобятся функции преобразователи их в прод надо не забыть
df_test = pd.read_json(test_path)

In [37]:
def test_data(df):
    df = df.T
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'user_id', 0: 'features'}, inplace=True)
    for key in unique_keys:
        df[key] = df['features'].apply(lambda x: x.get(key))
    df.drop('features', axis=1, inplace=True)
    df['orders_count'] = df['orders'].apply(count_site_ids_in_orders_modified)
    df['visits_count'] = df['visits'].apply(count_site_ids_in_visits_modified)
    df['last_visited_categories'] = df['last-visits-in-categories'].apply(extract_categories)
    df['site_meta_list'] = df['site-meta'].apply(extract_site_meta)
    df['item_ids_count'] = df['orders'].apply(count_item_ids_in_orders)
    df['category_path_count'] = df['orders'].apply(count_categories_in_orders)
    df['brand_ids_count'] = df['orders'].apply(count_brand_ids_in_orders)
    df['selected_sites'] = df['exchange-sessions'].apply(extract_selected_sites)
    df = df.drop(['visits', 'orders', 'last-visits-in-categories',
       'exchange-sessions', 'site-meta'], axis=1)
    return df

In [38]:
df_test = test_data(df_test)

In [39]:
df_test.head()

,user_id,orders_count,visits_count,last_visited_categories,site_meta_list,item_ids_count,category_path_count,brand_ids_count,selected_sites
0,user_155203,{},"{20: 3, 3: 5, 13: 3, 21: 1, 34: 3, 29: 1}","[hypermarket, other, media, electronics, cosme...","[21, 34, 20, 42, 33]",{},{},{},[13]
1,user_155204,{489: 1},"{161: 3, 30: 1, 20: 1, 206: 10, 46: 1, 153: 2,...","[insurance, furniture, fashion, other, bank, t...","[578, 98, 4, 17, 69, 398]",{},{},{},[]
2,user_155205,{},{3: 1},[other],[436],{},{},{},[]
3,user_155206,"{62: 2, 8: 1, 20: 8}","{3: 1, 62: 1, 13: 1, 46: 1, 8: 1, 206: 10, 16: 3}","[furniture, dealoftheday, fashion, other, bank...","[371, 591, 20, 104, 206, 62, 17, 8, 3, 400]","{'item_13288': 2, 'item_6173525': 2, 'item_855...","{91214: 2, 91197: 2, 90801: 2, 7070735: 2, 130...","{2914: 1, 129882: 1, 1927812: 1, 7184: 3, 4692...",[]
4,user_155207,"{104: 10, 719: 3, 50: 1}","{3: 1, 199: 3, 104: 10, 98: 1}","[other, furniture, shoes, cosmetics]","[50, 719, 104, 42]","{'item_189016': 1, 'item_651': 1, 'item_594': ...","{91179: 2, 13314794: 2, 91186: 2, 91157: 5, 90...","{124471: 9, 590: 5, 123682: 3}",[]


In [40]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18784 entries, 0 to 18783
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   user_id                  18784 non-null  object
 1   orders_count             18784 non-null  object
 2   visits_count             18784 non-null  object
 3   last_visited_categories  18784 non-null  object
 4   site_meta_list           18784 non-null  object
 5   item_ids_count           18784 non-null  object
 6   category_path_count      18784 non-null  object
 7   brand_ids_count          18784 non-null  object
 8   selected_sites           18784 non-null  object
dtypes: object(9)
memory usage: 1.3+ MB


## Второй этап извлечения интересного из данных, будем называть его - создание жадных статистических словарей

In [41]:
# Инициализация коэффициентов значимости для каждого столбца - пока подумаем может мы их по другому использовать
coefficients = {
    'orders_count_coeff': 1,
    'visits_count_coeff': 1,
    'last_visited_categories_coeff': 1,
    'site_meta_list_coeff': 1,
    'item_ids_count_coeff': 1,
    'category_path_count_coeff': 1,
    'brand_ids_count_coeff': 1,
    'selected_sites_coeff': 1
}

In [41]:
# стоит попробовать доверьтся данным грид серча. Но это пока не трогаем это один из экспериментов на будущее
#coefficients_greed = {
#    'orders_count_coeff': 1.5,
#    'visits_count_coeff': 1.5,
#    'last_visited_categories_coeff': 0.5,
#    'site_meta_list_coeff': 1,
#    'item_ids_count_coeff': 1.5,
#    'category_path_count_coeff': 0.5,
#    'brand_ids_count_coeff': 1.5,
#    'selected_sites_coeff': 0.5
#}

In [42]:
# Инициализация статистических словарей для каждого типа данных мы вытащим суть из данных в словари
orders_count_stats = {}
visits_count_stats = {}
last_visited_categories_stats = {}
site_meta_list_stats = {}
item_ids_count_stats = {}
category_path_count_stats = {}
brand_ids_count_stats = {}
selected_sites_stats = {}

In [43]:
# Проход по каждой строке DataFrame и мы заполним наши словари в логике ['ключ']:{male:0, female:0}
# по простому посчитаем с сколько сайт, предмет, категория встречается в данных у мужчин и женщин в нашем тренировочном наборе
# в персепктиве можно попробовать не считать count
for _, row in df_train.iterrows():
    gender = row['target']
    
    # Обработка orders_count
    for site_id, count in row['orders_count'].items():
        if site_id not in orders_count_stats:
            orders_count_stats[site_id] = {'male': 0, 'female': 0}
        orders_count_stats[site_id][gender] += count * coefficients['orders_count_coeff']

    # Обработка visits_count
    for site_id, count in row['visits_count'].items():
        if site_id not in visits_count_stats:
            visits_count_stats[site_id] = {'male': 0, 'female': 0}
        visits_count_stats[site_id][gender] += count * coefficients['visits_count_coeff']

    # Обработка last_visited_categories
    for category in row['last_visited_categories']:
        if category not in last_visited_categories_stats:
            last_visited_categories_stats[category] = {'male': 0, 'female': 0}
        last_visited_categories_stats[category][gender] += 1 * coefficients['last_visited_categories_coeff']

    # Обработка site_meta_list
    for site_id in row['site_meta_list']:
        if site_id not in site_meta_list_stats:
            site_meta_list_stats[site_id] = {'male': 0, 'female': 0}
        site_meta_list_stats[site_id][gender] += 1 * coefficients['site_meta_list_coeff']

    # Обработка item_ids_count
    for item_id, count in row['item_ids_count'].items():
        if item_id not in item_ids_count_stats:
            item_ids_count_stats[item_id] = {'male': 0, 'female': 0}
        item_ids_count_stats[item_id][gender] += count * coefficients['item_ids_count_coeff']

    # Обработка category_path_count
    for category_id, count in row['category_path_count'].items():
        if category_id not in category_path_count_stats:
            category_path_count_stats[category_id] = {'male': 0, 'female': 0}
        category_path_count_stats[category_id][gender] += count * coefficients['category_path_count_coeff']

    # Обработка brand_ids_count
    for brand_id, count in row['brand_ids_count'].items():
        if brand_id not in brand_ids_count_stats:
            brand_ids_count_stats[brand_id] = {'male': 0, 'female': 0}
        brand_ids_count_stats[brand_id][gender] += count * coefficients['brand_ids_count_coeff']

    # Обработка selected_sites
    for site_id in row['selected_sites']:
        if site_id not in selected_sites_stats:
            selected_sites_stats[site_id] = {'male': 0, 'female': 0}
        selected_sites_stats[site_id][gender] += 1 * coefficients['selected_sites_coeff']


Это уже третья версия программы, корректность записи статистических словарей мы тут уже не проверяем, но раньше проверяли и сохраняли и использовали для аналитики

## Третий этап создания жадных словарей - создание словарей признаков с доминантным полом

### Создание словарей с доминантными признаками

In [44]:
# Создание словарей для определения доминирующего пола для каждого типа данных (парадигма при равных побеждают
# мужчины, им нужна фора)
gender_dominant_orders_count = {
    site_id: 'female' if stats['female'] > stats['male'] else 'male'
    for site_id, stats in orders_count_stats.items()
}
gender_dominant_visits_count = {
    site_id: 'female' if stats['female'] > stats['male'] else 'male'
    for site_id, stats in visits_count_stats.items()
}
gender_dominant_last_visited_categories = {
    category: 'female' if stats['female'] > stats['male'] else 'male'
    for category, stats in last_visited_categories_stats.items()
}
gender_dominant_site_meta_list = {
    site_id: 'female' if stats['female'] > stats['male'] else 'male'
    for site_id, stats in site_meta_list_stats.items()
}
gender_dominant_item_ids_count = {
    item_id: 'female' if stats['female'] > stats['male'] else 'male'
    for item_id, stats in item_ids_count_stats.items()
}
gender_dominant_category_path_count = {
    category_id: 'female' if stats['female'] > stats['male'] else 'male'
    for category_id, stats in category_path_count_stats.items()
}
gender_dominant_brand_ids_count = {
    brand_id: 'female' if stats['female'] > stats['male'] else 'male'
    for brand_id, stats in brand_ids_count_stats.items()
}
gender_dominant_selected_sites = {
    site_id: 'female' if stats['female'] > stats['male'] else 'male'
    for site_id, stats in selected_sites_stats.items()
}

In [45]:
# создадим функцию для записи словаря в json но не будем ее использовать pickle сереализовать быстрее
def dict_to_json(dict_name, file_name):
    # Сохранение словаря в файл с использованием json
    with open(file_name + '.json', 'w') as file:
        json.dump(dict_name, file)

In [46]:
# создадим функцию для записи словаря в формат pickle
def dict_to_pickle(dict_name, file_name):
    # Сохранение словаря в файл с использованием pickle
    with open(file_name + '.pickle', 'wb') as file:
        pickle.dump(dict_name, file)

In [76]:
# Запишем эти словари, они могут нам пригодиться на проде
dict_to_pickle(gender_dominant_orders_count, 'gender_dominant_orders_count')
dict_to_pickle(gender_dominant_visits_count, 'gender_dominant_visits_count')
dict_to_pickle(gender_dominant_last_visited_categories, 'gender_dominant_last_visited_categories')
dict_to_pickle(gender_dominant_site_meta_list, 'gender_dominant_site_meta_list')
dict_to_pickle(gender_dominant_item_ids_count, 'gender_dominant_item_ids_count')
dict_to_pickle(gender_dominant_category_path_count, 'gender_dominant_category_path_count')
dict_to_pickle(gender_dominant_brand_ids_count, 'gender_dominant_brand_ids_count')
dict_to_pickle(gender_dominant_selected_sites, 'gender_dominant_selected_sites')

### Создание долевых словарей

Мы посчитаем доли посещения / покупок на сайтах и по категориям ну и вообще мужчин и женщин

In [47]:
# Заполнение словарей для расчета доли мужского и женского участия через фунцию
def calculate_gender_ratio(stats_dict):
    gender_ratio_dict = {}
    for key, stats in stats_dict.items():
        total = stats['male'] + stats['female']
        if total > 0:
            gender_ratio_dict[key] = {
                'male': round(stats['male'] / total, 10),
                'female': round(stats['female'] / total, 10)
            }
    return gender_ratio_dict

In [48]:
# Просто применим функцию
gender_ratio_orders_count = calculate_gender_ratio(orders_count_stats)
gender_ratio_visits_count = calculate_gender_ratio(visits_count_stats)
gender_ratio_last_visited_categories = calculate_gender_ratio(last_visited_categories_stats)
gender_ratio_site_meta_list = calculate_gender_ratio(site_meta_list_stats)
gender_ratio_item_ids_count = calculate_gender_ratio(item_ids_count_stats)
gender_ratio_category_path_count = calculate_gender_ratio(category_path_count_stats)
gender_ratio_brand_ids_count = calculate_gender_ratio(brand_ids_count_stats)
gender_ratio_selected_sites = calculate_gender_ratio(selected_sites_stats)


In [49]:
# Вывод первых пар элементов двух словарей для проверки - работает ну мы это и раньше проверяли
print(list(gender_ratio_orders_count.items())[:2])
print(list(gender_ratio_visits_count.items())[:2])

[(1, {'male': 0.6038330907, 'female': 0.3961669093}), (2, {'male': 0.5656984151, 'female': 0.4343015849})]
[(3, {'male': 0.4814309121, 'female': 0.5185690879}), (23, {'male': 0.7166733265, 'female': 0.2833266735})]


In [82]:
dict_to_pickle(gender_ratio_orders_count, 'gender_ratio_orders_count')
dict_to_pickle(gender_ratio_visits_count, 'gender_ratio_visits_count')
dict_to_pickle(gender_ratio_last_visited_categories, 'gender_ratio_last_visited_categories')
dict_to_pickle(gender_ratio_site_meta_list, 'gender_ratio_site_meta_list')
dict_to_pickle(gender_ratio_item_ids_count, 'gender_ratio_item_ids_count')
dict_to_pickle(gender_ratio_category_path_count, 'gender_ratio_category_path_count')
dict_to_pickle(gender_ratio_brand_ids_count, 'gender_ratio_brand_ids_count')
dict_to_pickle(gender_ratio_selected_sites, 'gender_ratio_selected_sites')

В целом на первый взгляд есть в этих правилах вполне понятная логика

А вот тут будет новая идея со значимостью словарей. Нас не устривает сильно нагручивать скор если данный пришли из словаря где 1 мужчина и 0 женщин, нас интересуют реально значимые словари где существено присутствие и мужчин и женщин и самое важное где есть сильный перекос к какую либо сторону

In [998]:
# создадим функцию, которая в каждом словаре найдет значимые элементы, базовые параметры от 1000 присутствией
# и перекос одного из значений в 1.5 раза (вот это мы будем очень продробно настравивать)
def find_significant_elements(gender_stats_dict, min_interactions=100, ratio_threshold=1.5):
    significant_elements = set()
    for element, stats in gender_stats_dict.items():
        total_interactions = stats['male'] + stats['female']
        if total_interactions > min_interactions:
            if max(stats['male'], stats['female']) / max(min(stats['male'], stats['female']), 1) >= ratio_threshold:
                significant_elements.add(element)
    return significant_elements

In [999]:
# Применение функции к каждому из словарей gender_stats что бы получить значимые позиции
significant_orders_count = find_significant_elements(orders_count_stats)
significant_visits_count = find_significant_elements(visits_count_stats)
significant_last_visited_categories = find_significant_elements(last_visited_categories_stats)
significant_site_meta_list = find_significant_elements(site_meta_list_stats)
significant_item_ids_count = find_significant_elements(item_ids_count_stats)
significant_category_path_count = find_significant_elements(category_path_count_stats)
significant_brand_ids_count = find_significant_elements(brand_ids_count_stats)
significant_selected_sites = find_significant_elements(selected_sites_stats)

In [1000]:
# Вывод полученных множеств значимых элементов для проверки
print("Significant Orders Count:", significant_orders_count)
print("Significant Visits Count:", significant_visits_count)
print("Significant Last Visited Categories:", significant_last_visited_categories)
print("Significant Site Meta List:", significant_site_meta_list)
print("Significant Item IDs Count:", significant_item_ids_count)
print("Significant Category Path Count:", significant_category_path_count)
print("Significant Brand IDs Count:", significant_brand_ids_count)
print("Significant Selected Sites:", significant_selected_sites)

Significant Orders Count: {1, 5, 6, 7, 8, 10, 11, 12, 15, 1224, 18, 23, 536, 25, 30, 31, 1057, 34, 548, 40, 1064, 42, 44, 45, 46, 556, 558, 1068, 50, 51, 565, 54, 55, 53, 57, 58, 62, 63, 65, 71, 73, 74, 76, 77, 590, 79, 80, 588, 82, 595, 84, 85, 86, 87, 88, 601, 89, 91, 92, 93, 606, 95, 96, 97, 98, 99, 100, 613, 94, 102, 104, 105, 106, 107, 108, 618, 110, 111, 114, 115, 117, 118, 629, 630, 122, 123, 124, 127, 128, 130, 644, 645, 646, 647, 135, 136, 650, 132, 137, 141, 653, 143, 655, 1163, 146, 147, 148, 149, 150, 151, 144, 153, 666, 155, 668, 156, 158, 159, 157, 162, 163, 134, 164, 165, 167, 1187, 678, 684, 173, 174, 175, 685, 688, 178, 179, 692, 181, 180, 183, 184, 182, 698, 185, 138, 648, 188, 139, 192, 193, 194, 195, 196, 197, 198, 199, 200, 140, 714, 202, 1220, 651, 206, 719, 208, 209, 1230, 205, 214, 726, 728, 216, 218, 727, 220, 1241, 734, 737, 225, 226, 1252, 234, 749, 752, 241, 244, 245, 758, 247, 248, 1271, 761, 254, 769, 257, 259, 260, 263, 775, 264, 266, 779, 268, 273, 275, 

In [1001]:
# Просто посмотрим на количество элементов много таких быть не должно
print("Significant Orders Count:", len(significant_orders_count), 'из общего количество', len(orders_count_stats.items()))
print("Significant Visits Count:", len(significant_visits_count), 'из общего количество', len(visits_count_stats.items()))
print("Significant Last Visited Categories:", len(significant_last_visited_categories), 'из общего количество', len(last_visited_categories_stats.items()))
print("Significant Site Meta List:", len(significant_site_meta_list), 'из общего количество', len(site_meta_list_stats.items()))
print("Significant Item IDs Count:", len(significant_item_ids_count), 'из общего количество', len(item_ids_count_stats.items()))
print("Significant Category Path Count:", len(significant_category_path_count), 'из общего количество', len(category_path_count_stats.items()))
print("Significant Brand IDs Count:", len(significant_brand_ids_count), 'из общего количество', len(brand_ids_count_stats.items()))
print("Significant Selected Sites:", len(significant_selected_sites), 'из общего количество', len(selected_sites_stats.items()))

Significant Orders Count: 294 из общего количество 902
Significant Visits Count: 263 из общего количество 476
Significant Last Visited Categories: 7 из общего количество 24
Significant Site Meta List: 337 из общего количество 836
Significant Item IDs Count: 3291 из общего количество 1146346
Significant Category Path Count: 1563 из общего количество 3321
Significant Brand IDs Count: 1770 из общего количество 32559
Significant Selected Sites: 147 из общего количество 585


# Переходим к предсказаниям

Вот с этой функцией можно поиграть что то отключить,  а может и что то добавить.

Сделаем небольшую заготовочку для проверки 2 функций

In [1002]:
df_check = pd.DataFrame()
df_check['id'] = df_val['user_id']
df_check['real_gender'] = df_val['target']
df_check.head()

,id,real_gender
0,user_127756,female
1,user_127757,male
2,user_127758,female
3,user_127759,female
4,user_127760,female


А вот собственно и этап проверки

In [1003]:
# функция которая считает точность
def calculate_accuracy(df, target, candidate):
    correct_predictions = df[df[target] == df[candidate]].shape[0]
    total_predictions = df.shape[0]
    accuracy = correct_predictions / total_predictions
    return accuracy


Тут мы будем работать со словарями значимости элемента которые мы готовили ранее, мы будем поощерять скор если он есть в значимых элементах и наказывать если нет

In [1045]:
# Создание столбца predict_gender учитывет значимость элемента
def predict_gender(row):
    male_score = 0
    female_score = 0
    significant_multiplier = 2.25  # Усиление значимого скора
    penalty_multiplier = 0.4    # Штраф незначимого скора
    coef = 1

    # Обработка orders_count
    for site_id, count in row['orders_count'].items():
        if site_id in significant_orders_count:
            coef = significant_multiplier 
        else:
            coef = penalty_multiplier
        if site_id in gender_ratio_orders_count:
            male_score += gender_ratio_orders_count[site_id]['male'] * count * coef 
            female_score += gender_ratio_orders_count[site_id]['female'] * count * coef

    # Обработка visits_count
    for site_id, count in row['visits_count'].items():
        if site_id in significant_visits_count:
            coef = significant_multiplier 
        else:
            coef = penalty_multiplier
        if site_id in gender_ratio_visits_count:
            male_score += gender_ratio_visits_count[site_id]['male'] * count * coef
            female_score += gender_ratio_visits_count[site_id]['female'] * count * coef

    # Обработка last_visited_categories - лучше убрать!
    #for category in row['last_visited_categories']:
    #    if category in significant_last_visited_categories:
    #        coef = significant_multiplier
    #    else:
    #        coef = penalty_multiplier
    #    if category in gender_ratio_last_visited_categories:
    #        male_score += gender_ratio_last_visited_categories[category]['male'] * coef 
    #        female_score += gender_ratio_last_visited_categories[category]['female'] * coef

    # Обработка site_meta_list
    for site_id in row['site_meta_list']:
        if site_id in significant_site_meta_list:
            coef = significant_multiplier * 3 # регистрация на сайте важный признак его нужно максить
        else:
            coef = penalty_multiplier * 8 # и не штрафовать даже если он не в списке
        if site_id in gender_ratio_site_meta_list:
            male_score += gender_ratio_site_meta_list[site_id]['male'] * coef
            female_score += gender_ratio_site_meta_list[site_id]['female'] * coef

    # Обработка item_ids_count
    for item_id, count in row['item_ids_count'].items():
        if site_id in significant_item_ids_count:
            coef = significant_multiplier
        else:
            coef = penalty_multiplier
        if item_id in gender_ratio_item_ids_count:
            male_score += gender_ratio_item_ids_count[item_id]['male'] * count * coef
            female_score += gender_ratio_item_ids_count[item_id]['female'] * count * coef

    # Обработка category_path_count - это тоже лучше убрать
    #for category_id, count in row['category_path_count'].items():
    #    if category_id in significant_category_path_count:
    #        coef = significant_multiplier
    #    else:
    #        coef = penalty_multiplier
    #    if category_id in gender_ratio_category_path_count:
    #        male_score += gender_ratio_category_path_count[category_id]['male'] * count * coef
    #        female_score += gender_ratio_category_path_count[category_id]['female'] * count * coef

    # Обработка brand_ids_count
    for brand_id, count in row['brand_ids_count'].items():
        if site_id in significant_brand_ids_count:
            coef = significant_multiplier
        else:
            coef = penalty_multiplier
        if brand_id in gender_ratio_brand_ids_count:
            male_score += gender_ratio_brand_ids_count[brand_id]['male'] * count * coef
            female_score += gender_ratio_brand_ids_count[brand_id]['female'] * count * coef

    # Обработка selected_sites
    #for site_id in row['selected_sites']:
    #    if site_id in significant_selected_sites:
    #        coef = significant_multiplier
    #    else:
    #        coef = penalty_multiplier
    #    if site_id in gender_ratio_selected_sites:
    #        male_score += gender_ratio_selected_sites[site_id]['male'] * coef
    #        female_score += gender_ratio_selected_sites[site_id]['female'] * coef

    return 'female' if female_score > male_score * 1.49 else 'male' # мужчины мало бегают по интернету поможем
                                                                    # коэффициентом

In [1046]:
# Применение функции к DataFrame
df_check['predict_gender'] = df_val.apply(predict_gender, axis=1)

In [1047]:
# Вызов функции для расчета accuracy для первого подхода
accuracy = calculate_accuracy(df_check, 'real_gender', 'predict_gender')
accuracy

0.8023463402193318

Мы проверили модель и на тренировочных данных для долевых словарей мы получили 0.83

In [1016]:
#второй вариант функции с доминантным полом
def predict_gender_dominant(row):
    male_score = 0
    female_score = 0
    significant_multiplier = 2  # Усиление значимого скора
    penalty_multiplier = 0.5    # Штраф незначимого скора
    coef = 1

    # Обработка orders_count
    for site_id in row['orders_count'].keys():
        if site_id in significant_orders_count:
            coef = significant_multiplier
        else:
            coef = penalty_multiplier
        if site_id in gender_dominant_orders_count:
            dominant_gender = gender_dominant_orders_count[site_id]
            if dominant_gender == 'male':
                male_score += 1 * coef
            else:
                female_score += 1 * coef

    # Обработка visits_count
    for site_id in row['visits_count'].keys():
        if site_id in significant_visits_count:
            coef = significant_multiplier
        else:
            coef = penalty_multiplier
        if site_id in gender_dominant_visits_count:
            dominant_gender = gender_dominant_visits_count[site_id]
            if dominant_gender == 'male':
                male_score += 1 * coef
            else:
                female_score += 1 * coef

    # Обработка last_visited_categories
    #for category in row['last_visited_categories']:
    #    if category in significant_last_visited_categories:
    #        coef = significant_multiplier
    #    else:
    #        coef = penalty_multiplier
    #    if category in gender_dominant_last_visited_categories:
    #        dominant_gender = gender_dominant_last_visited_categories[category]
    #        if dominant_gender == 'male':
    #            male_score += 1 * coef
    #        else:
    #            female_score += 1 * coef

    # Обработка site_meta_list
    for site_id in row['site_meta_list']:
        if site_id in significant_site_meta_list:
            coef = significant_multiplier 
        else:
            coef = penalty_multiplier
        if site_id in gender_dominant_site_meta_list:
            dominant_gender = gender_dominant_site_meta_list[site_id]
            if dominant_gender == 'male':
                male_score += 1 * coef
            else:
                female_score += 1 * coef

    # Обработка item_ids_count
    for item_id in row['item_ids_count'].keys():
        if item_id in significant_item_ids_count:
            coef = significant_multiplier
        else:
            coef = penalty_multiplier
        if item_id in gender_dominant_item_ids_count:
            dominant_gender = gender_dominant_item_ids_count[item_id]
            if dominant_gender == 'male':
                male_score += 1 * coef
            else:
                female_score += 1 * coef

    # Обработка category_path_count
    #for category_id in row['category_path_count'].keys():
    #    if category_id in gender_dominant_category_path_count:
    #        dominant_gender = gender_dominant_category_path_count[category_id]
    #        if dominant_gender == 'male':
    #            male_score += 1
    #        else:
    #            female_score += 1

    # Обработка brand_ids_count
    for brand_id in row['brand_ids_count'].keys():
        if brand_id in significant_brand_ids_count:
            coef = significant_multiplier
        else:
            coef = penalty_multiplier
        if brand_id in gender_dominant_brand_ids_count:
            dominant_gender = gender_dominant_brand_ids_count[brand_id]
            if dominant_gender == 'male':
                male_score += 1 * coef
            else:
                female_score += 1 * coef

    # Обработка selected_sites
    #for site_id in row['selected_sites']:
    #    if site_id in gender_dominant_selected_sites:
    #        dominant_gender = gender_dominant_selected_sites[site_id]
    #        if dominant_gender == 'male':
    #            male_score += 1
    #        else:
    #            female_score += 1

    return 'female' if female_score > male_score * 2 + 3 else 'male'

In [1014]:
# Применение функции к DataFrame
df_check['predict_gender_dominant'] = df_val.apply(predict_gender_dominant, axis=1)

In [1015]:
# Вызов функции для расчета accuracy для первого подхода
accuracy = calculate_accuracy(df_check, 'real_gender', 'predict_gender_dominant')
accuracy

0.774693044777207

Неплохо, но все же хуже чем предыдущий вариант с долевыми словарями, на тренировочном наборе точность - 0.81

In [1020]:
#второй вариант функции с доминантным полом
def predict_gender_dominant_personal(row):
    male_score = 0
    female_score = 0
    significant_multiplier = 2  # Усиление значимого скора
    penalty_multiplier = 0.3    # Штраф незначимого скора
    coef = 1

    # Обработка orders_count
    for site_id in row['orders_count'].keys():

        if site_id in gender_dominant_orders_count:
            dominant_gender = gender_dominant_orders_count[site_id]
            if dominant_gender == 'male':
                if site_id in significant_orders_count:
                    coef = significant_multiplier
                else:
                    coef = penalty_multiplier
                male_score += 1 * coef
            else:
                if site_id in significant_orders_count:
                    coef = significant_multiplier
                else:
                    coef = penalty_multiplier
                male_score += 1 * coef
                female_score += 1 * coef

    # Обработка visits_count
    for site_id in row['visits_count'].keys():
        
        if site_id in gender_dominant_visits_count:
            dominant_gender = gender_dominant_visits_count[site_id]
            if dominant_gender == 'male':
                if site_id in significant_visits_count:
                    coef = significant_multiplier
                else:
                    coef = penalty_multiplier
                male_score += 1 * coef
            else:
                if site_id in significant_visits_count:
                    coef = significant_multiplier
                else:
                    coef = penalty_multiplier
                female_score += 1 * coef

    # Обработка last_visited_categories
    for category in row['last_visited_categories']:
        
        if category in gender_dominant_last_visited_categories:
            dominant_gender = gender_dominant_last_visited_categories[category]
            if dominant_gender == 'male':
                if category in significant_last_visited_categories:
                    coef = significant_multiplier
                else:
                    coef = penalty_multiplier
                male_score += 1 * coef
            else:
                if category in significant_last_visited_categories:
                    coef = significant_multiplier
                else:
                    coef = penalty_multiplier
                female_score += 1 * coef

    # Обработка site_meta_list
    for site_id in row['site_meta_list']:
        
        if site_id in gender_dominant_site_meta_list:
            dominant_gender = gender_dominant_site_meta_list[site_id]
            if dominant_gender == 'male':
                if site_id in significant_site_meta_list:
                    coef = significant_multiplier
                else:
                    coef = penalty_multiplier
                male_score += 1 * coef
            else:
                if site_id in significant_site_meta_list:
                    coef = significant_multiplier
                else:
                    coef = penalty_multiplier
                female_score += 1 * coef

    # Обработка item_ids_count
    for item_id in row['item_ids_count'].keys():
        
        if item_id in gender_dominant_item_ids_count:
            dominant_gender = gender_dominant_item_ids_count[item_id]
            if dominant_gender == 'male':
                if item_id in significant_item_ids_count:
                    coef = significant_multiplier
                else:
                    coef = penalty_multiplier
                male_score += 1 * coef
            else:
                if item_id in significant_item_ids_count:
                    coef = significant_multiplier
                else:
                    coef = penalty_multiplier
                female_score += 1 * coef

    # Обработка category_path_count
    #for category_id in row['category_path_count'].keys():
    #    if category_id in gender_dominant_category_path_count:
    #        dominant_gender = gender_dominant_category_path_count[category_id]
    #        if dominant_gender == 'male':
    #            male_score += 1
    #        else:
    #            female_score += 1

    # Обработка brand_ids_count
    for brand_id in row['brand_ids_count'].keys():
        
        if brand_id in gender_dominant_brand_ids_count:
            dominant_gender = gender_dominant_brand_ids_count[brand_id]
            if dominant_gender == 'male':
                if brand_id in significant_brand_ids_count:
                    coef = significant_multiplier
                else:
                    coef = penalty_multiplier
                male_score += 1 * coef
            else:
                if brand_id in significant_brand_ids_count:
                    coef = significant_multiplier
                else:
                    coef = penalty_multiplier
                female_score += 1 * coef

    # Обработка selected_sites
    #for site_id in row['selected_sites']:
    #    if site_id in gender_dominant_selected_sites:
    #        dominant_gender = gender_dominant_selected_sites[site_id]
    #        if dominant_gender == 'male':
    #            male_score += 1
    #        else:
    #            female_score += 1

    return 'female' if female_score > male_score * 2 + 1 else 'male'

In [1018]:
# Применение функции к DataFrame
df_check['predict_gender_dominant_personal'] = df_val.apply(predict_gender_dominant_personal, axis=1)

In [1019]:
# Вызов функции для расчета accuracy для первого подхода
accuracy = calculate_accuracy(df_check, 'real_gender', 'predict_gender_dominant_personal')
accuracy

0.7896309250555616

In [962]:
df_check_train = pd.DataFrame()
df_check_train['id'] = df_train['user_id']
df_check_train['real_gender'] = df_train['target']
df_check_train.head()

,id,real_gender
0,user_1,female
1,user_2,female
2,user_3,male
3,user_4,male
4,user_5,female


In [963]:
# Применение функции к DataFrame
df_check_train['predict_gender'] = df_train.apply(predict_gender, axis=1)
# Вызов функции для расчета accuracy для первого подхода
accuracy = calculate_accuracy(df_check_train, 'real_gender', 'predict_gender')
accuracy

0.8336503463660914

In [566]:
# Применение функции к DataFrame
df_check_train['predict_gender_dominant'] = df_train.apply(predict_gender_dominant, axis=1)
# Вызов функции для расчета accuracy для первого подхода
accuracy = calculate_accuracy(df_check_train, 'real_gender', 'predict_gender_dominant')
accuracy

0.8202653516496419

In [567]:
# Применение функции к DataFrame
df_check_train['predict_gender_dominant_personal'] = df_train.apply(predict_gender_dominant_personal, axis=1)
# Вызов функции для расчета accuracy для первого подхода
accuracy = calculate_accuracy(df_check_train, 'real_gender', 'predict_gender_dominant_personal')
accuracy

0.815905444013933

# Анализ ошибок тоже может помочь!) займемся им позже

In [43]:
# Создание DataFrame с ошибочными предсказаниями
df_errors = df_transposed_val[df_transposed_val['target'] != df_transposed_val['predict_gender']]

In [44]:
# Вывод первых нескольких строк DataFrame с ошибочными предсказаниями
df_errors.head()

,user_id,target,exchange-sessions,orders,visits,site-meta,last-visits-in-categories,orders_count,visits_count,last_visited_categories,predict_gender
8,user_127764,male,None,"[{'site-id': 179, 'orders': [{'created-at': 13...","[{'site-id': 3, 'first-seen': 1690804257, 'las...","[{'site-id': 179, 'recency': 1, 'frequency': 2...","[{'category': 'fashion', 'last-visit-at': 1685...","{179: 2, 40: 1}","{3: 2, 13: 1, 2: 1, 208: 2, 85: 2, 431: 6, 7: 1}","[fashion, other, bank, household_appliances, f...",female
9,user_127765,female,None,"[{'site-id': 16, 'orders': [{'created-at': 156...","[{'site-id': 3, 'first-seen': 1684519666, 'las...","[{'site-id': 2}, {'site-id': 16, 'recency': 1,...","[{'category': 'other', 'last-visit-at': 169651...","{16: 1, 2: 6, 691: 2}",{3: 4},[other],male
14,user_127770,female,None,"[{'site-id': 39, 'orders': [{'created-at': 167...","[{'site-id': 13, 'first-seen': 1684836326, 'la...","[{'site-id': 2}, {'site-id': 39, 'recency': 1,...","[{'category': 'bank', 'last-visit-at': 1674136...","{39: 1, 244: 1, 1: 1}","{13: 1, 3: 1}","[bank, electronics, other, media, cosmetics, f...",male
15,user_127771,male,"[{'landed-at': 1698678911, 'sites': [169, 2, 2...","[{'site-id': 62, 'orders': [{'created-at': 168...","[{'site-id': 34, 'first-seen': 1685510491, 'la...","[{'site-id': 42}, {'site-id': 33}, {'site-id':...","[{'category': 'fashion', 'last-visit-at': 1698...","{62: 1, 40: 1, 209: 2, 29: 5, 74: 1}","{34: 2, 135: 2, 149: 1, 26: 4, 110: 2, 713: 1,...","[fashion, other, bank, electronics, cosmetics,...",female
20,user_127776,male,"[{'landed-at': 1697833017, 'sites': [2, 169, 7...","[{'site-id': 49, 'orders': [{'created-at': 150...","[{'site-id': 912, 'first-seen': 1688017479, 'l...","[{'site-id': 495}, {'site-id': 49, 'recency': ...","[{'category': 'other', 'last-visit-at': 169783...","{49: 10, 191: 1, 34: 1, 112: 6, 21: 1, 39: 1, ...","{912: 2, 495: 5, 66: 1, 178: 1, 357: 1, 51: 1,...","[other, bank, travel, pets, food, electronics,...",female
